# Exporting the DataFrame to MySQL Database

## Importing the dependencies

In [ ]:
import mysql.connector as ms
from mysql.connector import MySQLConnection
import pandas as pd
import csv

## Establishing a connection with MySQL Database

In [ ]:
db = MySQLConnection(user = 'root',
                     host = 'localhost',
                     password = 'yourpassword',
                     database = 'database_name')

In [ ]:
cursor = db.cursor(buffered = True)

## Importing the DataFrame

In [ ]:
df_books = pd.read_csv('Books.csv')
df_books.head()

In [ ]:
for i in df_books:
    print(i)

In [ ]:
for i in df_books:
    print(i, ":", sum(df_books[i].isna()))

In [ ]:
df_books.dropna(inplace = True)

## Creating the Table

In [ ]:
books = """
CREATE TABLE books(ISBN VARCHAR(255) NOT NULL,
Book_Title VARCHAR(255) NOT NULL,
Book_Author VARCHAR(255) NOT NULL,
Year_Of_Publication INT(4) NOT NULL,
Publisher VARCHAR(255) NOT NULL,
Image_URL_S VARCHAR(255) NOT NULL,
Image_URL_M VARCHAR(255) NOT NULL,
Image_URL_L VARCHAR(255) NOT NULL);
"""

In [ ]:
cursor.execute(books)

In [ ]:
cursor.execute("SHOW TABLES;")
cursor.fetchall()

## Adding Values to the Tables

In [ ]:
book_values = df_books.values

In [ ]:
for i in book_values:
    print(i)

In [ ]:
for i in range(0, len(book_values)):
    cursor.execute("""
    INSERT INTO books(ISBN, Book_Title, Book_Author, Year_Of_Publication, Publisher, Image_URL_S, Image_URL_M, Image_URL_L)
    VALUES(%s,%s,%s,%s,%s,%s,%s,%s)
    """,list(book_values[i]))
    print(i)

## Adding the rest of the tables to the database

In [ ]:
df_users = pd.read_csv("Users.csv")
df_ratings = pd.read_csv("Ratings.csv")

In [ ]:
for i in df_users:
    print(i)

In [ ]:
for i in df_ratings:
    print(i)

In [ ]:
df_users['User-ID'].value_counts()

In [ ]:
df_ratings['User-ID'].value_counts()

In [ ]:
df_ratings.head()

In [ ]:
cursor.execute('''CREATE TABLE users(User_ID VARCHAR(255),
Location VARCHAR(255),
Age VARCHAR(255));''')
cursor.execute('''CREATE TABLE ratings(User_ID VARCHAR(255),
ISBN VARCHAR(255),
Book_Rating int(2));''')

In [ ]:
cursor.execute("SHOW TABLES;")
cursor.fetchall()

In [ ]:
ratings = df_ratings.values

In [ ]:
for i in range(0, len(ratings)):
    cursor.execute("""
    INSERT INTO ratings(USER_ID, ISBN, Book_Rating)
    VALUES(%s,%s,%s)
    """,list(ratings[i]))
    print(i)

In [ ]:
sum(df_users['Age'].isna())

In [ ]:
df_users['Age'].fillna(-1, inplace = True)

In [ ]:
users

In [ ]:
users = df_users.values
for i in range(0, len(users)):
    cursor.execute("""
    INSERT INTO users(USER_ID, Location, Age)
    VALUES(%s,%s,%s)
    """,list(users[i]))
    print(i)

In [ ]:
cursor.execute("SELECT * FROM ratings;")

In [ ]:
t = (cursor.fetchall())

In [ ]:
t